## Reading Data From MongoDB ##

In [1]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
from python_speech_features import sigproc
from scipy.fftpack import dct
import os
from pymongo import MongoClient
from sklearn.preprocessing import StandardScaler

In [2]:
# connecting to MongoDB
client= MongoClient('mongodb://localhost:27017')
mongo_database= client['AudioFeatures']
collection_name= mongo_database['features_collection']
read_data= collection_name.find()
data_list= list(read_data)
audio_df= pd.DataFrame(data_list)

## Data Preparation ##

In [5]:
audio_df['flat_features']= audio_df['mfcc_features'].apply(lambda x: [item for sublist in x for item in sublist])
min_length= audio_df['flat_features'].apply(len).min()
audio_df['truncated_features']= audio_df['flat_features'].apply(lambda x: x[:min_length])
mfcc_array= np.array(audio_df['truncated_features'].tolist())
# standardize the data
scaler= StandardScaler()
mfcc_scaled= scaler.fit_transform(mfcc_array)
audio_df['scaled_features']= list(mfcc_scaled)

In [7]:
for index, row in audio_df.iterrows():
    # update each document of db with new scaled features
    collection_name.update_one(
        {'_id': row['_id']},
        {'$set': {'scaled_features': row['scaled_features'].tolist()}}
    )